<a href="https://colab.research.google.com/github/ehdgnsdl/dacon/blob/main/penguin_updated1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

from os.path import join
import pandas as pd
import numpy as np
import multiprocessing

n_cpus = multiprocessing.cpu_count()

ROOT_PATH = ''
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/penguin/train.csv')
x_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/penguin/test.csv')

In [ ]:
data.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400
3,3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.20106,-26.16524,4850
4,4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.38324,-26.84272,5700


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   114 non-null    int64  
 1   Species              114 non-null    object 
 2   Island               114 non-null    object 
 3   Clutch Completion    114 non-null    object 
 4   Culmen Length (mm)   114 non-null    float64
 5   Culmen Depth (mm)    114 non-null    float64
 6   Flipper Length (mm)  114 non-null    int64  
 7   Sex                  111 non-null    object 
 8   Delta 15 N (o/oo)    111 non-null    float64
 9   Delta 13 C (o/oo)    111 non-null    float64
 10  Body Mass (g)        114 non-null    int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 9.9+ KB


In [ ]:
data['Flipper Length (mm)'] = data['Flipper Length (mm)'].astype('float')
data['Flipper Length (mm)'].dtype

dtype('float64')

In [ ]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   228 non-null    int64  
 1   Species              228 non-null    object 
 2   Island               228 non-null    object 
 3   Clutch Completion    228 non-null    object 
 4   Culmen Length (mm)   228 non-null    float64
 5   Culmen Depth (mm)    228 non-null    float64
 6   Flipper Length (mm)  228 non-null    float64
 7   Sex                  222 non-null    object 
 8   Delta 15 N (o/oo)    219 non-null    float64
 9   Delta 13 C (o/oo)    220 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 17.9+ KB


In [ ]:
label = data['Body Mass (g)']

data.drop(columns=['id', 'Body Mass (g)'], inplace=True)
x_test.drop(columns=['id'], inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t =='O']
num_columns = [c for c in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n\n Numeric Columns: \n{}\n'.format(cat_columns, num_columns))

Categorical Columns: 
['Species', 'Island', 'Clutch Completion', 'Sex']

 Numeric Columns: 
['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)']



In [ ]:
# data 수치형 변수들의 결측치 확인
pd.isna(data[num_columns]).sum().sum()

6

In [ ]:
# x_test 수치형 변수들의 결측치 확인
pd.isna(x_test[num_columns]).sum().sum()

17

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(data, label,
                                                      test_size = 0.3,
                                                      random_state=42,
                                                      shuffle=True)
x_train.reset_index(drop=True, inplace=True)
x_valid.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

mice = IterativeImputer(random_state=0)
x_train[num_columns] = mice.fit_transform(x_train[num_columns])
x_valid[num_columns] = mice.transform(x_valid[num_columns])
x_test[num_columns] = mice.transform(x_test[num_columns])

In [ ]:
pd.isna(x_train[num_columns]).sum().sum()

0

In [ ]:
pd.isna(x_valid[num_columns]).sum().sum()

0

In [ ]:
pd.isna(x_test[num_columns]).sum().sum()

0

In [ ]:
# 범주형 변수 결측치 처리 및 라벨인코딩
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

ohe = OneHotEncoder(sparse=False)
ohe.fit(data[cat_columns])

OneHotEncoder(sparse=False)

In [ ]:
onehot_columns = ohe.categories_[0].tolist()
for col in ohe.categories_[1:]:
  onehot_columns += col.tolist()

In [ ]:
x_train_cat = pd.DataFrame(ohe.transform(x_train[cat_columns]), columns=onehot_columns)
x_valid_cat = pd.DataFrame(ohe.transform(x_valid[cat_columns]), columns=onehot_columns)
x_test_cat  = pd.DataFrame(ohe.transform(x_test[cat_columns]),  columns=onehot_columns)

In [ ]:
# .values를 해줌으로써 밑에 Xgboost에서 featuer_names is unique라는 에러를 해결
x_train = pd.concat([x_train[num_columns], x_train_cat], axis=1)
x_valid = pd.concat([x_valid[num_columns], x_valid_cat], axis=1)
x_test  = pd.concat([x_test[num_columns],  x_test_cat],  axis=1)

In [ ]:
y_train.shape

(79,)

In [ ]:
# 라벨 변수 Label Encoding
le = LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [ ]:
y_valid

80     3900
4      5700
40     5250
69     5350
10     4550
45     4800
70     4650
66     5650
47     4725
11     3300
98     5500
36     5000
83     4700
112    2700
18     4650
0      5550
72     4500
26     4400
81     3900
53     3800
103    5350
91     4075
12     4850
31     3450
67     6300
111    4100
108    4300
22     4650
44     3800
107    3650
78     5400
15     5000
30     3750
104    5000
9      4750
Name: Body Mass (g), dtype: int64

In [ ]:
# 스케일링
from sklearn.preprocessing import StandardScaler
sclaer = StandardScaler()
x_train[num_columns] = sclaer.fit_transform(x_train[num_columns])
x_valid[num_columns] = sclaer.transform(x_valid[num_columns])
x_test[num_columns] = sclaer.transform(x_test[num_columns])

In [ ]:
# 스케일링 확인 (std인 표준편차가 1에 가까워야함)
x_train[num_columns].describe()

,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01
mean,5.621382e-18,-1.319620e-15,4.314411e-16,8.432074e-16,1.023092e-15
std,1.006390e+00,1.006390e+00,1.006390e+00,1.006390e+00,1.006390e+00
min,-1.948239e+00,-1.980462e+00,-1.906274e+00,-1.991752e+00,-1.559992e+00
25%,-8.526895e-01,-7.281967e-01,-7.871240e-01,-8.470913e-01,-7.869291e-01
50%,7.714643e-02,6.405284e-02,-2.974957e-01,-2.042377e-01,-2.972046e-01
75%,8.780942e-01,7.796331e-01,8.566282e-01,9.924208e-01,8.066664e-01
max,2.028881e+00,2.006342e+00,2.010752e+00,2.280513e+00,1.903624e+00


In [ ]:
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 16 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Culmen Length (mm)                         79 non-null     float64
 1   Culmen Depth (mm)                          79 non-null     float64
 2   Flipper Length (mm)                        79 non-null     float64
 3   Delta 15 N (o/oo)                          79 non-null     float64
 4   Delta 13 C (o/oo)                          79 non-null     float64
 5   Adelie Penguin (Pygoscelis adeliae)        79 non-null     float64
 6   Chinstrap penguin (Pygoscelis antarctica)  79 non-null     float64
 7   Gentoo penguin (Pygoscelis papua)          79 non-null     float64
 8   Biscoe                                     79 non-null     float64
 9   Dream                                      79 non-null     float64
 10  Torgersen                   

In [ ]:
# 모델 검증 + OOF 앙상블
from sklearn.model_selection import KFold
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
# 회귀 모델 사용해보기
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

val_scores = list()
#oof_pred = np.zeros((x_test.shape[0], label.shape[0])

models = [['Linear Regression', LinearRegression(n_jobs=n_cpus)],
          ['MultiLayerPerceptron', MLPRegressor(random_state=42)],
          ['RandomForest', RandomForestRegressor(random_state=42,
                                                 n_jobs=n_cpus)],
          ['XGBoost', XGBRegressor(random_state=42,
                                   tree_method='gpu_hist',
                                   n_jobs=n_cpus)],
          ['LightGBM', LGBMRegressor(random_state=42,
                                     tree_method='gpu_hist',
                                     n_jobs=n_cpus)]]

best_loss = np.inf
best_pred = None
for i, (name, model) in enumerate(models):
  current_loss = list()
  current_pred = list()
  
  model.fit(x_train, y_train)
  y_pred = model.predict(x_valid)
  current_loss.append(RMSE(y_valid, y_pred))
  current_pred.append(model.predict(x_test))

  if np.mean(current_loss) < best_loss:
      best_loss  = np.mean(current_loss)
      best_model = [i, name]
      best_pred  = current_pred
      #oof_pred += model.predict(x_test) / n_splits
    
  print(f'{name} Model Validation RMSE: {np.mean(current_loss)}')

print(f'Best Model: {best_model[-1]}, RMSE: {best_loss}')

Linear Regression Model Validation RMSE: 345.59930641040074
MultiLayerPerceptron Model Validation RMSE: 4632.256876832222
RandomForest Model Validation RMSE: 441.50552698368665
[04:31:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Model Validation RMSE: 386.85679306184494
LightGBM Model Validation RMSE: 403.621426772355
Best Model: Linear Regression, RMSE: 345.59930641040074


In [ ]:
# best_pred를 np.array()로 만든 후에 (list를 array로 만들어줌.)
best_pred = np.array(best_pred)
best_pred.shape

(1, 228)

In [ ]:
best_pred = best_pred.reshape(-1, 1)
best_pred.shape

(228, 1)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/penguin/sample_submission.csv')
submit.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
submit['Body Mass (g)'] = best_pred

In [ ]:
submit_file_name = 'penguin_second.csv'
submit.to_csv(submit_file_name, index=False)